In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys
sys.path.append('/home/hrai/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
joint_2ds = {'train': [], 'test': []}
confidences = {'train': [], 'test': []}
joint3d_images = {'train': [], 'test': []}
joints_25d_images = {'train': [], 'test': []}
_25d_factors = {'train': [], 'test': []}
camera_names = {'train': [], 'test': []}
actions = {'train': [], 'test': []}
sources = {'train': [], 'test': []}
frames = {'train': [], 'test': []}
world_3ds = {'train': [], 'test': []}
cam_3ds = {'train': [], 'test': []}
cam_params = {'train': [], 'test': []}

# target 
pkl_folder = '/home/hrai/codes/MotionBERT/data/motion3d/3dhp_gt_test_pkl'
test_subject = ['TS1', 'TS2', 'TS3', 'TS4', 'TS5', 'TS6']
only_test = True

blacklist = []

for item in os.listdir(pkl_folder):
    print(item)
    subject = item.split('.')[0]
    
    # filtering
    #if action not in action_list: continue 
    #if int(cam_num) not in camera_list: continue
    if only_test:
        if subject not in test_subject: continue
    if subject in blacklist: continue
    #print(subject, cam_num, action, phase) 
    
    item_path = os.path.join(pkl_folder, item) 
    data = readpkl(item_path) 
    if subject in test_subject: data_type = 'test' 
    else: data_type = 'train' 
    joint_2ds[data_type] += list(np.array(data['joint_2d'])) 
    confidences[data_type] += list(np.array(data['confidence'])) 
    camera_names[data_type] += list(data['camera_name'])
    actions[data_type] += list(data['action'])
    sources[data_type] += list(data['source'])
    frames[data_type] += list(data['frame'])
    cam_3ds[data_type] += list(np.array(data['cam_3d'])/1000.0) # mm to m
    print(cam_3ds[data_type][0])
    
data_total = {'train': {}, 'test': {}}

for data_type in ['train', 'test']:
    data_total[data_type]['joint_2d']          = copy.deepcopy(np.array(joint_2ds[data_type]))
    data_total[data_type]['confidence']        = copy.deepcopy(np.array(confidences[data_type]))
    data_total[data_type]['camera_name']       = copy.deepcopy(np.array(camera_names[data_type]))
    data_total[data_type]['action']            = copy.deepcopy(np.array(actions[data_type]))
    data_total[data_type]['source']            = copy.deepcopy(np.array(sources[data_type]))
    data_total[data_type]['frame']             = copy.deepcopy(np.array(frames[data_type]))
    data_total[data_type]['cam_3d']            = copy.deepcopy(np.array(cam_3ds[data_type]))


if (len(data_total['train']['joint_2d']) == 0) or only_test:
    for key in data_total['train'].keys():
        data_total['train'][key] = copy.deepcopy(data_total['test'][key][:243])
        print(key, data_total['train'][key].shape)
data_total['train']['joint_2d'].shape, data_total['test']['joint_2d'].shape  

TS4.pkl
[[-4.485      -0.0828665   1.45249   ]
 [-4.37874    -0.07643998  1.3652875 ]
 [-4.30678     0.373488    1.66474   ]
 [-4.41822     0.626292    1.38108   ]
 [-4.59126    -0.08929302  1.5396925 ]
 [-4.37523    -0.0187427   1.95478   ]
 [-4.18878     0.0373882   2.29957   ]
 [-4.55461939 -0.31571267  1.35964271]
 [-4.55897    -0.598212    1.3578    ]
 [-4.52808    -0.690315    1.37819   ]
 [-4.496      -0.88629     1.37872   ]
 [-4.68328    -0.543012    1.45183   ]
 [-4.8036     -0.271953    1.59232   ]
 [-4.83691    -0.0557766   1.7071    ]
 [-4.43782    -0.531715    1.25626   ]
 [-4.32077    -0.225296    1.2467    ]
 [-4.40479    -0.0371808   1.38297   ]]
TS5.pkl
[[-4.485      -0.0828665   1.45249   ]
 [-4.37874    -0.07643998  1.3652875 ]
 [-4.30678     0.373488    1.66474   ]
 [-4.41822     0.626292    1.38108   ]
 [-4.59126    -0.08929302  1.5396925 ]
 [-4.37523    -0.0187427   1.95478   ]
 [-4.18878     0.0373882   2.29957   ]
 [-4.55461939 -0.31571267  1.35964271]
 [-4.558

((243, 17, 2), (24888, 17, 2))

In [3]:
data_total['train']['joint_2d'].shape, data_total['train']['confidence'].shape  

((243, 17, 2), (243, 17, 1))

In [4]:
save_path = os.path.join('/home/hrai/codes/MotionBERT', f'data/motion3d/3dhp_gt_test.pkl')
savepkl(data_total, save_path)